<font color = "red"> Binana close contacts! 

#### Overview 

| Task                                      | Description                                                   |
|-------------------------------------------|---------------------------------------------------------------|
| Run Binana                                | Execute Binana for all PDBs and poses                         |
| Read logfile                              | Create a dictionary with information from log files           |
| Getting dict info                         | - Close Contacts [OK] <br> - Hydrogen Bonds                  |
| Extract atom properties (RDkit)           | - Atom Index <br> - Name <br> - Mass <br> - Charge <br> - Element Name <br> - Hybridization <br> - Num Hydrogens <br> - Formal Charge <br> - Unpaired Electron <br> - In Aromatic Substructure |
| Preparing data for graph                  | - protgraph_dict[pdb][pose] = {<br>&nbsp;&nbsp;&nbsp;&nbsp;'x_s': x_s (ligand properties),<br>&nbsp;&nbsp;&nbsp;&nbsp;'x_t': x_t (receptor properties),<br>&nbsp;&nbsp;&nbsp;&nbsp;'edge_index': edge_index (Matrix [2, number of connections]),<br>&nbsp;&nbsp;&nbsp;&nbsp;'edge_attr': edge_attr (Vector [distances])<br>} |
| Protgraph_dict                            | --> BipartiteData data  --> Dataset list  --> DataLoader  --> Graph Attention Model |


* Data spliting methods

### Libraries 

In [3]:
from tqdm import tqdm
import pickle
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import subprocess
import numpy as np
import torch
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
import os
from torch_geometric.loader import DataLoader
from pathlib import Path
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import GCNConv
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from collections import Counter
from torch_geometric.data import Data, Dataset
import json
import os
import pandas as pd
import pandas as pd
from rdkit import Chem
import torch
import numpy as np
from torch.utils.data import Dataset
import pickle

/home/lbcb02/anaconda3/envs/master_37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%run Config.ipynb

In [5]:
config = Config()

In [6]:
pdbs = os.listdir("coreset")#[0:10]
nposes = config.docking_params['nposes']
batch_size  = config.model_args['batch_size']
path_scripts = Path(os.getcwd())
num_epochs = config.model_args['epochs']
num_folds = config.model_args['nfolds']
patience = config.model_args['patience']  # Set your desired patience value

### Running Binana
<font color = "red"> Só precisa rodar uma vez!

In [5]:
%%time
def binana(pdb, config):
    binana_executable = "/Softwares/binana-2.1/python/run_binana.py"
    
    ligand_path = os.path.join(path_scripts, config['ligand_path'])
    output_folder = os.path.join(path_scripts, config['output_folder'])
    receptor_path = os.path.join(path_scripts, config['receptor_path'])
    
    os.makedirs(output_folder, exist_ok=True)  # Create the directory path if it doesn't exist
    
    binana_exec_cmd = f"python3 {binana_executable} -receptor {receptor_path} -ligand {ligand_path} -output_dir {output_folder} > errors.txt"
    os.system(binana_exec_cmd)


# Combined loop for both versions
for pdb in tqdm(pdbs, desc="Processing PDBS"):

    config_version_2 = {
    'ligand_path': f"coreset/{pdb}/ligand.pdbqt",
    'output_folder': f"/home/lbcb02/Workspace/Scripts/coreset/{pdb}/binana/binana_{pdb}",
    'receptor_path': f"coreset/{pdb}/receptor.pdbqt"
}
    
    for pose in range(1, nposes + 1):
        # Configuration dictionaries for each version
        config_version_1 = {
            'ligand_path':f"coreset/{pdb}/results/pose_{pose}.pdbqt",
            'output_folder': f"/home/lbcb02/Workspace/Scripts/coreset/{pdb}/binana/binana_{pdb}_pose_{pose}",
            'receptor_path': f"coreset/{pdb}/receptor.pdbqt"
        }
        try:
            binana(pdb, config=config_version_1)
        except ValueError as e:
            print(e)
            pass
    binana(pdb, config=config_version_2)

Processing PDBS:   5%|█▏                       | 13/285 [01:36<38:29,  8.49s/it]Traceback (most recent call last):
  File "/Softwares/binana-2.1/python/run_binana.py", line 13, in <module>
    start()
  File "/Softwares/binana-2.1/python/run_binana.py", line 8, in start
    binana.run()
  File "/Softwares/binana-2.1/python/binana/__init__.py", line 97, in run
    _start._get_all_interactions(cmd_params)
  File "/Softwares/binana-2.1/python/binana/_start.py", line 48, in _get_all_interactions
    ligand, receptor = from_files(
                       ^^^^^^^^^^^
  File "/Softwares/binana-2.1/python/binana/load_ligand_receptor/__init__.py", line 98, in from_files
    ligand.load_pdb_file(ligand_filename)
  File "/Softwares/binana-2.1/python/binana/_structure/mol.py", line 239, in load_pdb_file
    file = openFile(filename, "r")
           ^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/home/lbcb02/Workspace/Scripts/coreset/4mme/results/pose_9.pdbqt'
Trace

CPU times: user 1.17 s, sys: 612 ms, total: 1.78 s
Wall time: 26min 9s


### Read logfile

In [7]:
%%time
from collections import defaultdict
from pathlib import Path

data_dict_all_logs_exp = defaultdict(dict)
data_dict_all_logs_docked = defaultdict(dict)
def read_logfile(pdb, pose=None):
    # Define the common part of the file path
    common_path = f"{path_scripts}/coreset/{pdb}/binana/binana_{pdb}"
    
    # Adjust the file path based on whether the pose parameter is provided
    path = common_path + (f"_pose_{pose}" if pose is not None else "") + "/log.txt"

     
    try: # Read the text file using Path
        with Path(path).open('r') as file:
            text = file.read()
    
    
        # Extract the content after "JSON Output:"
        json_text = text.split("JSON Output:")[1].strip()
        
        # Convert the JSON text to a dictionary
        data_dict = json.loads(json_text)
        
        # Store the data dictionary for the pose if pose is provided
        if pose is not None:
            data_dict_all_logs_docked[pdb][pose] = data_dict
        else:
            # Store the data dictionary for the pose (from the first code)
            data_dict_all_logs_exp[pdb] = data_dict

    except:
        print("Error when opening file ", path)
    
    return data_dict_all_logs_exp, data_dict_all_logs_docked

# Example usage: provide pose parameter for the second code, leave it empty for the first code
for pdb in pdbs:
    read_logfile(pdb)  # Run logic from the first code
    for pose in range(1, nposes + 1):
        read_logfile(pdb, pose)  # Run logic from the second code

Error when opening file  /home/lbcb02/Workspace/Scripts/coreset/4mme/binana/binana_4mme_pose_9/log.txt
Error when opening file  /home/lbcb02/Workspace/Scripts/coreset/4mme/binana/binana_4mme_pose_10/log.txt
Error when opening file  /home/lbcb02/Workspace/Scripts/coreset/3u8n/binana/binana_3u8n_pose_7/log.txt
Error when opening file  /home/lbcb02/Workspace/Scripts/coreset/3u8n/binana/binana_3u8n_pose_8/log.txt
Error when opening file  /home/lbcb02/Workspace/Scripts/coreset/3u8n/binana/binana_3u8n_pose_9/log.txt
Error when opening file  /home/lbcb02/Workspace/Scripts/coreset/3u8n/binana/binana_3u8n_pose_10/log.txt
Error when opening file  /home/lbcb02/Workspace/Scripts/coreset/4j3l/binana/binana_4j3l_pose_9/log.txt
Error when opening file  /home/lbcb02/Workspace/Scripts/coreset/4j3l/binana/binana_4j3l_pose_10/log.txt
Error when opening file  /home/lbcb02/Workspace/Scripts/coreset/3fur/binana/binana_3fur_pose_8/log.txt
Error when opening file  /home/lbcb02/Workspace/Scripts/coreset/3fur/b

### Getting dict info 

In [8]:
%%time
def flatten_data(df, pdb, pose = False):
    flat_data = []
    if pose == False:
        data = df[pdb]['closeContacts']
    else:
        data = df[pdb][pose]['closeContacts']
    
    for entry in data:
        flat_entry = {
            'ligand_atom_index': entry['ligandAtoms'][0]['atomIndex'],
            'ligand_atom_name': entry['ligandAtoms'][0]['atomName'],
            'ligand_chain': entry['ligandAtoms'][0]['chain'],
            'ligand_resID': entry['ligandAtoms'][0]['resID'],
            'ligand_resName': entry['ligandAtoms'][0]['resName'],
            #'angle': entry['metrics']['angle'],
            'distance': entry['metrics']['distance'],
            'receptor_atom_index': entry['receptorAtoms'][0]['atomIndex'],
            'receptor_atom_name': entry['receptorAtoms'][0]['atomName'],
            'receptor_chain': entry['receptorAtoms'][0]['chain'],
            'receptor_resID': entry['receptorAtoms'][0]['resID'],
            'receptor_resName': entry['receptorAtoms'][0]['resName'],
        }
        flat_data.append(flat_entry)

    return pd.DataFrame(flat_data)


CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


In [ ]:
data_dict_all_logs_docked

In [26]:
data_dict_all_logs_docked.get("4eor")

{1: {'activeSiteFlexibility': {'BACKBONE_BETA': 28,
   'BACKBONE_OTHER': 2,
   'SIDECHAIN_BETA': 66,
   'SIDECHAIN_OTHER': 3},
  'cationPiInteractions': [],
  'closeContacts': [{'ligandAtoms': [{'atomIndex': 1,
      'atomName': 'C2',
      'chain': 'd',
      'resID': 0,
      'resName': 'UNN'}],
    'metrics': {'distance': 3.5374903533437387},
    'receptorAtoms': [{'atomIndex': 114,
      'atomName': 'CG2',
      'chain': 'A',
      'resID': 10,
      'resName': 'ILE'}]},
   {'ligandAtoms': [{'atomIndex': 1,
      'atomName': 'C2',
      'chain': 'd',
      'resID': 0,
      'resName': 'UNN'}],
    'metrics': {'distance': 3.918316475222491},
    'receptorAtoms': [{'atomIndex': 1291,
      'atomName': 'CD2',
      'chain': 'A',
      'resID': 134,
      'resName': 'LEU'}]},
   {'ligandAtoms': [{'atomIndex': 3,
      'atomName': 'N1',
      'chain': 'd',
      'resID': 0,
      'resName': 'UNN'}],
    'metrics': {'distance': 3.659126398472729},
    'receptorAtoms': [{'atomIndex': 114,

In [11]:
data_frames_exp = [
    flatten_data(data_dict_all_logs_exp, pdb).assign(pdb=pdb)
    for pdb in pdbs
]
data_frame_exp = pd.concat(data_frames_exp, axis=0)

#data_frames_docked = [flatten_data(data_dict_all_logs_docked, pdb, pose).assign(pdb=pdb, pose=str(pose))  for pdb in pdbs  for pose in range(1, nposes)]
#data_frame_docked = pd.concat(data_frames_docked, axis=0)

In [16]:
%%time
# Create a list of DataFrames with the specified conditions
data_frames_docked = [
    flatten_data(data_dict_all_logs_docked, pdb, pose).assign(pdb=pdb, pose=str(pose))
    for pdb in pdbs
    for pose in range(1, nposes)
    if pose in data_dict_all_logs_docked.get(pdb, {})
]

# Concatenate the DataFrames
data_frame_docked = pd.concat(data_frames_docked, axis=0)

In [30]:
df = data_frame_docked.copy()

In [31]:
df['pdb'].nunique()

285

#### List of indexes sorted 

Here we create 2 dicts ids_lig, ids_rec that will have the atoms indices. It is necessary, so we know from what atoms we should obtains 
the properties.

<font color = "red"> Why i used process_ids to sort the values 

In [33]:
%%time
def process_ids(data_frame, pdb,  ligand_column, pose = False):
    if 'pose' in data_frame.columns:
        aux = list(data_frame[(data_frame['pdb'] == pdb) & (data_frame['pose'] == str(pose))][ligand_column])
    else:
        aux = list(data_frame[data_frame['pdb'] == pdb][ligand_column])
    try:
        aux = [int(val) for val in aux]
    except ValueError:
        pass
    return aux

def process_pdb_data(pdbs, data_frame, ligand_column):
    ids = {}
    for pdb in pdbs:
        if 'pose' in data_frame.columns:
            ids[pdb] = {}
            for pose in range(1, nposes + 1):
                ids[pdb][pose] = process_ids(data_frame, pdb,  ligand_column, pose)
        else:
            ids[pdb] = process_ids(data_frame, pdb, ligand_column)
    return ids

# index
ids_lig_with_poses = process_pdb_data(pdbs, data_frame_docked, 'ligand_atom_index')
ids_rec_with_poses  = process_pdb_data(pdbs, data_frame_docked, 'receptor_atom_index')
ids_lig_exp = process_pdb_data(pdbs, data_frame_exp, 'ligand_atom_index')
ids_rec_exp  = process_pdb_data(pdbs, data_frame_exp, 'receptor_atom_index')

## Names
#names_lig_with_poses = process_pdb_data(pdbs, data_frame_docked, 'ligand_atom_name')
#names_rec_with_poses  = process_pdb_data(pdbs, data_frame_docked, 'receptor_atom_name')
#names_lig_exp = process_pdb_data(pdbs, data_frame_exp, 'ligand_atom_name')
#names_rec_exp  = process_pdb_data(pdbs, data_frame_exp, 'receptor_atom_name')

CPU times: user 3min 33s, sys: 55.9 ms, total: 3min 33s
Wall time: 3min 33s


In [12]:
data_frame.head(2)

,pdb,pose,ligand_atom_index,ligand_atom_name,receptor_atom_index,receptor_atom_name,distance
0,4eor,1,1,C2,114,CG2,3.537490
1,4eor,1,1,C2,1291,CD2,3.918316


In [35]:
%%time
def compare_graphs(pairs_graph1, pairs_graph2):
    common_pairs = [pair for pair in pairs_graph1 if pair in pairs_graph2]
    print(len(common_pairs))
    similarity_score = (len(common_pairs) / len(pairs_graph2)) * 100
    return similarity_score

score = {}

for pdb in pdbs:
    score[pdb] = {}
    for pose in range(1, nposes + 1):
        pairs_graph1 = list(zip(ids_lig_with_poses[pdb][pose], ids_rec_with_poses[pdb][pose]))
        pairs_graph2 = list(zip(ids_lig_exp[pdb], ids_rec_exp[pdb]))
        score[pdb][pose] = compare_graphs(pairs_graph1, pairs_graph2)

import json 
with open("Data/score.json", "w") as json_file:
    json.dump(score, json_file)

16
82
8
7
20
3
2
8
2
0
82
101
75
73
75
77
79
83
20
0
111
115
27
68
15
16
1
40
6
0
55
4
9
3
5
32
4
1
18
0
27
21
23
0
7
10
11
2
6
0
51
3
42
3
4
47
1
18
16
0
21
86
29
12
12
8
2
10
26
0
78
49
1
37
35
25
20
11
22
0
53
28
35
38
22
46
27
37
30
0
30
50
56
48
47
42
56
50
77
0
125
75
24
24
24
17
48
0
33
0
115
93
81
67
36
75
16
5
11
0
21
20
2
5
11
29
4
11
27
0
78
18
7
2
19
5
2
0
0
0
93
108
86
65
38
40
12
2
8
0
159
80
21
34
16
18
29
35
4
0
95
66
17
1
70
0
4
0
5
0
121
17
20
20
11
16
3
9
7
0
95
58
0
7
12
7
18
4
7
0
85
14
68
14
13
7
38
7
8
0
151
70
24
2
87
24
3
21
14
0
121
78
118
32
96
55
16
6
45
0
136
123
38
31
12
39
3
33
3
0
64
112
97
47
33
64
32
75
32
0
124
10
0
27
1
13
11
17
22
0
4
11
41
5
22
27
13
11
3
0
3
9
44
1
2
16
2
11
1
0
49
13
27
15
42
3
11
16
3
0
9
131
16
84
67
72
11
8
10
0
3
2
68
47
55
0
35
3
0
0
90
70
7
7
9
0
0
0
0
0
3
8
7
141
5
144
5
147
5
0
65
3
63
74
2
3
0
5
0
0
27
8
42
11
13
10
9
0
8
0
67
21
27
13
11
1
6
7
32
0
1
14
14
6
2
3
74
6
16
0
23
29
8
5
11
18
2
0
2
0
70
94
21
30
13
24
8
5
15

<font color = "yellow"> When i extrated the atom properties, i only extrated for  ids_3gv9_pose_1

Why am i taking this?

     * Because we only want properties from the important nodes

### Extract atom properties

In [13]:
def find_element_index(lst, element):
    """
    Locate the index of the desired element within the list. If the element is not found, return the index of the last item.
    """
    try:
        return lst.index(element)
    except ValueError:
        return len(lst) - 1

In [14]:
class AtomPropertiesExtractor:
    def __init__(self):
        #self.ligand_path = ligand_path
        self.columns = ["Atom_Index", "Name", "Mass", "Charge", "Element_Name", "Hybridization", "Num_Hydrogens", "Formal_Charge", "Unpaired_Electron", "In_Aromatic_Substructure"]
        self.allowable_features = {
            "possible_hybridization_list": ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "UNSPECIFIED"],
            "possible_atoms": ['C', 'S', 'N', 'O', 'H']
        }

    def extract_properties(self, atom_indices, properties_to_extract, path):
        atom_properties_list = []

        with open(path, "r") as pdb_file:
            pdb_data = pdb_file.read()

        molecule = Chem.MolFromPDBFile(path, removeHs=False)

        if molecule is not None:
            for atom_index in atom_indices:
                atom = molecule.GetAtomWithIdx(atom_index - 1)

                atom_properties = {
                    "Atom_Index": atom_index,
                    "Name": atom.GetSymbol(),
                    "Mass": atom.GetMass(),
                    "Charge": atom.GetFormalCharge(),
                    "Element_Name": find_element_index(self.allowable_features['possible_atoms'], str(atom.GetSymbol())),
                    "Hybridization": find_element_index(self.allowable_features['possible_hybridization_list'], str(atom.GetHybridization())),
                    "Num_Hydrogens": atom.GetTotalNumHs(),
                    "Formal_Charge": atom.GetFormalCharge(),
                    "Unpaired_Electron": atom.GetNumRadicalElectrons(),
                    "In_Aromatic_Substructure": int(atom.GetIsAromatic())
                }

                atom_properties = {key: atom_properties[key] for key in properties_to_extract}
                atom_properties_list.append(atom_properties)

        atom_properties_df = pd.DataFrame(atom_properties_list, columns=properties_to_extract)
        return atom_properties_df

In [15]:
# Create an instance of the class
atom_ligand_extractor = AtomPropertiesExtractor()
atom_receptor_extractor =  AtomPropertiesExtractor()

In [16]:
#atom_receptor_extractor.extract_properties(ids_lig['3gv9'][2], node_descriptors, ligand_path.as_posix())

#### Dict with atom properties: Ligand and receptor

In [17]:
#extract_atom_properties(ligand_path.as_posix(), ids_lig['3gv9'][2])

In [18]:
atom_ligand_extractor.extract_properties(ids_lig['3gv9'][1], config.node_descriptors, '/home/lbcb02/Workspace/Scripts/coreset/3gv9/results/pose_1.pdb')

,Mass,Charge,Element_Name,Hybridization,Num_Hydrogens,Formal_Charge,Unpaired_Electron,In_Aromatic_Substructure
0,12.011,0,0,2,0,0,0,1
1,12.011,0,0,2,1,0,0,1
2,12.011,0,0,2,1,0,0,1
3,32.067,0,1,2,0,0,0,1
4,12.011,0,0,2,0,0,0,1
5,14.007,0,2,2,0,0,0,0
6,12.011,0,0,2,0,0,0,0
7,15.999,0,3,2,0,0,0,0
8,12.011,0,0,3,3,0,0,0
9,1.008,0,4,0,0,0,0,0


In [19]:
ids_lig['3gv9'][2]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [20]:
lig_atoms_prop = {}
rec_atoms_prop = {}
df_dict = {}

In [21]:
%%time
for pdb in tqdm(pdbs, desc = "Processing PDBs"):
    lig_atoms_prop[pdb] = {}
    rec_atoms_prop[pdb] = {}
    df_dict[pdb] = {}
    print(pdb)
    for pose in range(1, nposes+1):
        ligand_path =  path_scripts/ f"coreset/{pdb}/results/pose_{pose}.pdb"
        protein_path = path_scripts/ f"coreset/{pdb}/{pdb}_protein.pdb"
       
        lig_atoms_prop[pdb][pose] = atom_ligand_extractor.extract_properties(ids_lig_with_poses[pdb][pose], config.node_descriptors, ligand_path.as_posix())#.drop(columns = 'Name')
        rec_atoms_prop[pdb][pose] = atom_receptor_extractor.extract_properties(ids_rec_with_poses[pdb][pose], config.node_descriptors, protein_path.as_posix())#.drop(columns = 'Name')
            
        df_dict[pdb][pose] = df[(df['pdb']==pdb) & (df['pose']==str(pose))]
        #print(rec_atoms_prop[pdb][pose].shape)
        #print(lig_atoms_prop[pdb][pose].shape)

Processing PDBs:   0%|                                   | 0/54 [00:00<?, ?it/s]

3gv9


Processing PDBs:   2%|▌                          | 1/54 [00:00<00:31,  1.68it/s][15:10:03] Explicit valence for atom # 5109 N, 5, is greater than permitted
[15:10:03] Explicit valence for atom # 5109 N, 5, is greater than permitted
[15:10:03] Explicit valence for atom # 5109 N, 5, is greater than permitted
[15:10:03] Explicit valence for atom # 5109 N, 5, is greater than permitted
[15:10:03] Explicit valence for atom # 5109 N, 5, is greater than permitted


3gr2


[15:10:03] Explicit valence for atom # 5109 N, 5, is greater than permitted
[15:10:03] Explicit valence for atom # 5109 N, 5, is greater than permitted
[15:10:03] Explicit valence for atom # 5109 N, 5, is greater than permitted
[15:10:03] Explicit valence for atom # 5109 N, 5, is greater than permitted
[15:10:03] Explicit valence for atom # 5109 N, 5, is greater than permitted
Processing PDBs:   4%|█                          | 2/54 [00:01<00:26,  1.97it/s]

...

2qbp


Processing PDBs:  98%|█████████████████████████▌| 53/54 [00:38<00:00,  1.54it/s]

3b68


Processing PDBs: 100%|██████████████████████████| 54/54 [00:39<00:00,  1.38it/s]

CPU times: user 38.7 s, sys: 269 ms, total: 39 s
Wall time: 39.1 s


<font color = red> Should i keep L, R? 

In [22]:
#lig_atoms_prop['Atom_Index'] = "L" + lig_atoms_prop['Atom_Index'].astype(str)
#rec_atoms_prop['Atom_Index'] = "R" + rec_atoms_prop['Atom_Index'].astype(str)

### Preparing data for graph

Obs: For 3gr2 pose 10 we have nothing! 

In [24]:
#torch.tensor(np.array(lig_atoms_prop['3gr2'][10].iloc[:, 1:].astype("float32")))

In [25]:
#torch.tensor(np.array(rec_atoms_prop['3gr2'][pose].iloc[:, 1:].astype("float32")))

<font color ="red ">x_t tensor([], size=(0, 8)) for 3gr2!

#### Label 

Obs: "Error calculating RMSD for 4kz6 pose 1: No sub-structure match found between the reference
and probe mol"
<font color = 'red'> For  now, i will assume the label as 0 to just to begin!

In [26]:
label = pd.read_json("/home/lbcb02/Workspace/Scripts/master/Data/label.json").fillna(0)

In [27]:
label.head()

,3gv9,3gr2,4llx,2xdl,4kz6,2v00,2hb1,3nq9,2wbg,4jxs,...,2qbq,3rlr,2xb8,1u1b,3prs,4pcs,1g2k,2al5,2qbp,3b68
1,0,0,1,1,0.0,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,0,0,1,1,1.0,1,1,0,1,1,...,1,0,1,1,1,1,1,1,1,1
3,1,0,0,0,0.0,1,1,0,0,1,...,1,0,0,1,0,1,0,1,1,0
4,0,0,0,0,0.0,0,1,0,1,1,...,1,0,0,1,0,0,0,0,1,0
5,1,0,1,0,1.0,0,0,0,1,1,...,0,0,0,1,0,1,0,0,0,0
6,0,0,1,0,1.0,0,0,0,0,1,...,1,0,0,1,0,1,0,0,0,0
7,1,0,1,0,1.0,0,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0
8,0,0,1,1,0.0,1,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
9,1,0,0,0,1.0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
10,0,1,0,0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
len(label.columns)

54

In [33]:
# #******************************** PSG-BAR  Altered ********************************
# class Get_dataset(Dataset):
#     def __init__(self):
#        print("init")
#     def get_unique_prot_graphs(self):
#         #calculate protein graphs for all unique pdbs in the data
        
#         self.protgraph_dict = dict()
#         self.failed_protgraph =[]
        
#         for pdb in pdbs:
#             self.protgraph_dict[pdb] = {}  # Initialize an empty dictionary for each pdb
#             for pose in range(1, nposes + 1):
               
#                 x_s = torch.tensor(np.array(lig_atoms_prop[pdb][pose].iloc[:, 1:].astype("float32")))
#                 x_t = torch.tensor(np.array(rec_atoms_prop[pdb][pose].iloc[:, 1:].astype("float32")))
#                 edge_index = torch.tensor(np.array([df_dict[pdb][pose]["ligand_atom_index"].astype("int64"), df_dict[pdb][pose]["receptor_atom_index"].astype("int64")]))
#                 edge_index = map_node_names_to_indices(edge_index)
#                 distances = torch.tensor(np.array(df_dict[pdb][pose]['distance']).astype("float32"))
            
#                 self.protgraph_dict[pdb][pose]= {'x_s':x_s,
#                                                  'x_t':x_t,
#                                                  'edge_index':edge_index,
#                                                  'edge_attr':distances,
#                                                 }

#         return self.protgraph_dict
        
#     def __len__(self):
#         # Return the total number of data samples in your dataset
#         return len(self.protgraph_dict)  # You may need to modify this depending on your data structure

#     def __get__(self, index):
#         pdb = list(self.protgraph_dict.keys())[index]
#         pose = 1  # You may need to modify this
#         data_sample = self.protgraph_dict[pdb][pose]
#         return data_sample

        

In [30]:
class Get_dataset():
    def __init__(self, pdbs, lig_atoms_prop, rec_atoms_prop, df_dict, subsample=None, preprocessed_protgraph=False):
        self.pdbs = pdbs
        self.lig_atoms_prop = lig_atoms_prop
        self.rec_atoms_prop = rec_atoms_prop
        self.df_dict = df_dict
        self.protgraph_dict = self.get_unique_prot_graphs()
        self.preprocessed_protgraph=preprocessed_protgraph
        self.subsample = subsample 
        
    def map_node_names_to_indices(edge_index):
        unique_node_names_0, node_name_to_index_0 = torch.unique(edge_index[0], sorted=True, return_inverse=True)
        unique_node_names_1, node_name_to_index_1 = torch.unique(edge_index[1], sorted=True, return_inverse=True)
    
        # Map the node names to their corresponding indices separately for edge_index[0] and edge_index[1]
        updated_edge_index_0 = node_name_to_index_0.view(1, -1)
        updated_edge_index_1 = node_name_to_index_1.view(1, -1)
    
        # Combine the updated edge indices back into the same format as edge_index
        updated_edge_index = torch.cat((updated_edge_index_0, updated_edge_index_1), dim=0)
    
        return updated_edge_index
        
    def get_unique_prot_graphs(self):
        protgraph_dict = {}
        failed_protgraph = []

        for pdb in self.pdbs:
            protgraph_dict[pdb] = {}
            for pose in range(1, nposes + 1):
                x_s = torch.tensor(np.array(self.lig_atoms_prop[pdb][pose].iloc[:, 1:].astype("float32")))
                x_t = torch.tensor(np.array(self.rec_atoms_prop[pdb][pose].iloc[:, 1:].astype("float32")))
                edge_index = torch.tensor(np.array([self.df_dict[pdb][pose]["ligand_atom_index"].astype("int64"),
                                                    self.df_dict[pdb][pose]["receptor_atom_index"].astype("int64")]))
                edge_index = map_node_names_to_indices(edge_index)
                distances = torch.tensor(np.array(self.df_dict[pdb][pose]['distance']).astype("float32"))

                protgraph_dict[pdb][pose] = {'x_s': x_s,
                                             'x_t': x_t,
                                             'edge_index': edge_index,
                                             'edge_attr': distances,
                                             }
        return protgraph_dict


    def process(self):
        print('processessing....')
        if self.preprocessed_protgraph:
            with open(f'{config.root}/{config.project_name}/protgraph_dict.pkl','rb') as f:
               protgraph_dict = pickle.load(f)
        else:
            protgraph_dict = self.get_unique_prot_graphs()
            with open(f'{config.root}/{config.project_name}/protgraph_dict.pkl','wb') as f:
                pickle.dump(protgraph_dict,f)
         
        for pdb in pdbs:
            for pose in range(1, nposes + 1): 

                protgraph = self.protgraph_dict[pdb][pose] ### Needs alteration to open saved dict, lets got with it righ now

                interaction_id = str(pdb) + "_" + str(pose)                 
     
                x_s, x_t, edge_index, edge_attr= protgraph['x_s'], protgraph['x_t'], protgraph['edge_index'],protgraph['edge_attr']

                #y = torch.randint(2, (1,), dtype=torch.float32)
                try:
                    y = torch.tensor(label[pdb][pose], dtype=torch.float32)
                except Exception as e:
                    print(f"Error: {e}")
                    continue
                #y = label[pdb][pose].astype("float32")
                data = BipartiteData(edge_index=edge_index, x_s=x_s, x_t=x_t, y=y, edge_attr = edge_attr)
                print(f'saving data {config.processed_dir}')
                torch.save(data, os.path.join(config.processed_dir, f'data_{interaction_id}.pt'))    

    
    def processed_file_names(self):
        datalist =   [torch.load(os.path.join(config.processed_dir, f'{filename}')) for filename in os.listdir(config.processed_dir)]
        return datalist

   # def __len__(self):
   #     return len(protgraph_dict)  ###
   # 
   #def __getitem__(self, index):
   #    pdb = self.pdbs[index]
   #    pose = 1  # You may need to modify this
   #    data_sample = self.protgraph_dict[pdb][pose]
   #    return data_sample

  

dataset = Get_dataset(pdbs, lig_atoms_prop, rec_atoms_prop, df_dict)

In [31]:
dataset.get_unique_prot_graphs()['3gv9'][1]['edge_index'].shape

torch.Size([2, 57])

In [ ]:
#dataset.get_unique_prot_graphs()['3gv9'][1]['edge_attr'].shape

#### BipartiteData 

In [36]:
class BipartiteData(Data):
    def __init__(self, edge_index=None, x_s=None, x_t=None, y=None, edge_attr=None):
        super().__init__()
        self.edge_index = edge_index
        self.x_s = x_s
        self.x_t = x_t
        self.y = y
        self.edge_attr = edge_attr  # Add edge_attr attribute
        self.num_nodes = (x_s.size(0) if x_s is not None else 0) + (x_t.size(0) if x_t is not None else 0)

    
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index':
            return torch.tensor([[self.x_s.size(0)], [self.x_t.size(0)]])
        else:
            return super().__inc__(key, value, *args, **kwargs)

#### Dataset

In [37]:
%%time
dataset.process()
dataset_list = dataset.processed_file_names()

processessing....
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving data .//master/processed_dir
saving dat

In [40]:
dataset_list

[BipartiteData(edge_index=[2, 128], x_s=[24, 7], x_t=[48, 7], y=0.0, edge_attr=[128], num_nodes=72),
 BipartiteData(edge_index=[2, 101], x_s=[19, 7], x_t=[43, 7], y=0.0, edge_attr=[101], num_nodes=62),
 BipartiteData(edge_index=[2, 57], x_s=[16, 7], x_t=[32, 7], y=0.0, edge_attr=[57], num_nodes=48),
 BipartiteData(edge_index=[2, 58], x_s=[15, 7], x_t=[23, 7], y=0.0, edge_attr=[58], num_nodes=38),
 BipartiteData(edge_index=[2, 0], x_s=[0, 7], x_t=[0, 7], y=0.0, edge_attr=[0], num_nodes=0),
 BipartiteData(edge_index=[2, 192], x_s=[44, 7], x_t=[83, 7], y=1.0, edge_attr=[192], num_nodes=127),
 BipartiteData(edge_index=[2, 59], x_s=[25, 7], x_t=[28, 7], y=0.0, edge_attr=[59], num_nodes=53),
 BipartiteData(edge_index=[2, 195], x_s=[19, 7], x_t=[65, 7], y=0.0, edge_attr=[195], num_nodes=84),
 BipartiteData(edge_index=[2, 127], x_s=[27, 7], x_t=[50, 7], y=0.0, edge_attr=[127], num_nodes=77),
 BipartiteData(edge_index=[2, 56], x_s=[21, 7], x_t=[25, 7], y=1.0, edge_attr=[56], num_nodes=46),
 Bip

In [38]:
len(dataset_list)

540

### Saving pkl 

In [194]:
# Save the list to a file
with open(f'{config.data}/bipartite_data.pkl', 'wb') as file:
    pickle.dump(dataset_list, file)

In [37]:
with open(f'{config.data}/bipartite_data.pkl', 'rb') as file:
    dataset_list = pickle.load(file)

In [38]:
len(dataset_list)

540

In [60]:
##### Filter data list 

len(dataset_list)
filtered_data_list_num_nodes = [data for data in dataset_list if data.num_nodes>0]
filtered_data_list_descriptors = [data for data in filtered_data_list_num_nodes if data.x_s.shape[0] > 0 and data.x_t.shape[0] > 0]
len(filtered_data_list)

400

In [61]:
label_distribuition = dict(Counter([label.y.tolist() for label in filtered_data_list]))
amount_of_graphs_used_to_train = len(filtered_data_list)

## Dataset info 

In [65]:
print(5*"*******")
print("Dataset 2016: 285")
print("Docked complexes: ", len(pdbs))
print("Nº of poses generated: ", len(dataset_list))
print("Nº graphs with features: ", len(filtered_data_list_num_nodes))
print("Nº Dataset to train ", len(filtered_data_list))
print(5*"*******")

***********************************
Dataset 2016: 285
Docked complexes:  54
Nº of poses generated:  540
Nº graphs with features:  486
Nº Dataset to train  400
***********************************


| Priority | Status      | Task Description                                |
|----------|-------------|-------------------------------------------------|
| 1        | To Do       | -   label                              |
|          |             |   - Problem of rmsd                            |
|          |             |   - Put rmsd method: rdkit                     |
| 1        | To Do       | - Model descriptors                            |
|          |             |   - Nodes meaning                                |
| 2        | To Do       | - Add more metrics                             |
| 2        | To Do       | - Print more metrics                           |
| 3        | To Do       | - Cross-validation method as info              |
| 3        | To Do       | - Revise split methods                         |
| 4        | To Do       | - See what binna considers as close contact    |
| 5        | To Do       | - Clean code                                   |
| 5        | To Do       | - Comment code                                 |
| 5        | To Do       | - Process all data                             |


##### Figure 3: Cumulative fraction of results with crystal-like (within 2 Å RMSD) binding mode plotted versus pose number as ranked

Cumulative fraction of systems that contain at least one correct pose up to a given pose number.

    For example, at x=5, the y-values indicate the fraction of systems that have at least one 
    “positive” pose in the top 5 according to the rankings specified by each model. 
    
    It can be seen that the “L+LP+R” model maintains roughly the advantage seen on the first pose until approximately x=10, where all plots in Figure 3 start to level off. In 65% of systems in our test set, the docking program samples a correct mode in at least one of the 20 rank positions.